In [1]:
# Let's first read the CSV file to understand its structure and data, and then we can proceed with merging if necessary.
import pandas as pd

# Load the uploaded CSV file
csv_file_path = 'data-GtKMn.csv'
csv_data = pd.read_csv(csv_file_path)

# Display the first few rows of the CSV file to understand its structure
csv_data.head()

,X.100,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree,Graduate or professional degree
0,Capitol Region Council of Governments,8.34%,26.10%,23.84%,23.04%,18.68%
1,Connecticut Metropolitan Council of Governments,11.61%,25.70%,21.80%,22.61%,18.28%
2,Lower Connecticut River Valley Council of Gove...,4.40%,24.66%,24.79%,24.28%,21.87%
3,Naugatuck Valley Council of Governments,11.23%,29.70%,27.76%,17.44%,13.86%
4,Northeastern Connecticut Council of Governments,6.89%,36.27%,31.36%,14.75%,10.73%


In [2]:

import geopandas as gpd

# Load the uploaded geojson file
geojson_file_path = 'Connecticut_Planning_Region_Index.geojson'
geo_data = gpd.read_file(geojson_file_path)

# Display the first few rows of the geojson file to understand its structure
geo_data.head()

,OBJECTID,RPO_COD,REGPLANORG,SHORT_NAME,geometry
0,15,1,Capitol Region Council of Governments,CRCOG,"POLYGON ((-72.81668 42.00011, -72.81684 41.997..."
1,16,2,Connecticut Metropolitan Council of Governments,CTMetro,"POLYGON ((-73.18062 41.38913, -73.18021 41.388..."
2,17,3,Lower Connecticut River Valley Council of Gove...,RiverCOG,"POLYGON ((-72.49953 41.63697, -72.49236 41.625..."
3,18,4,Naugatuck Valley Council of Governments,Naugatuck Valley,"POLYGON ((-72.89822 41.72317, -72.89323 41.698..."
4,19,5,Northeastern Connecticut Council of Governments,NECCOG,"POLYGON ((-72.16835 42.03071, -72.15021 42.030..."


In [3]:
# Let's inspect the raw contents of the geojson file to identify any issues with the geometry or structure.
with open(geojson_file_path, 'r') as file:
    geojson_content = file.read()

# Display the first 500 characters of the file to inspect the structure
geojson_content[:500]

'{"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"EPSG:4326"}},"features":[{"type":"Feature","id":15,"geometry":{"type":"Polygon","coordinates":[[[-72.816681357862,42.0001062447778],[-72.8168363760937,41.9977136454365],[-72.8147249878562,41.9979115633585],[-72.8085662469324,41.9985804547488],[-72.8032023807912,41.9991562157734],[-72.7993765357768,41.9996002251911],[-72.797588525943,41.9997808819823],[-72.7947224745261,42.0001053141457],[-72.7930206757759,42.0003050757506],[-'

In [4]:
# Let's try a different approach to load the GeoJSON using an alternate method.
from shapely.geometry import shape
import json

# Load the GeoJSON data manually
with open(geojson_file_path, 'r') as f:
    geojson_data = json.load(f)

# Extract features from the GeoJSON
features = geojson_data['features']

# Convert features to a GeoDataFrame manually
geo_df = pd.DataFrame([{
    'id': feature['id'],
    'geometry': shape(feature['geometry']),
    **feature['properties']
} for feature in features])

# Display the first few rows of the manually created GeoDataFrame
geo_df.head()


,id,geometry,OBJECTID,RPO_COD,REGPLANORG,SHORT_NAME
0,15,"POLYGON ((-72.816681357862 42.0001062447778, -...",15,1,Capitol Region Council of Governments,CRCOG
1,16,"POLYGON ((-73.1806159340512 41.3891266711625, ...",16,2,Connecticut Metropolitan Council of Governments,CTMetro
2,17,"POLYGON ((-72.4995323283692 41.6369683595543, ...",17,3,Lower Connecticut River Valley Council of Gove...,RiverCOG
3,18,"POLYGON ((-72.8982245346013 41.7231736702374, ...",18,4,Naugatuck Valley Council of Governments,Naugatuck Valley
4,19,"POLYGON ((-72.1683514817699 42.0307094815158, ...",19,5,Northeastern Connecticut Council of Governments,NECCOG


In [5]:
# First, let's rename the first column in the CSV data for clarity, and then perform the merge based on the region names.
csv_data.rename(columns={'X.1': 'REGPLANORG'}, inplace=True)

# Perform the merge based on the 'REGPLANORG' column
merged_data = pd.merge(geo_df, csv_data, on='REGPLANORG', how='inner')

merged_data


KeyError: 'REGPLANORG'

In [ ]:
# Convert the merged data back into a GeoDataFrame
merged_geo_data = gpd.GeoDataFrame(merged_data, geometry='geometry')

# Save the GeoDataFrame as a new GeoJSON file
output_geojson_path = 'merged_data.geojson'
merged_geo_data.to_file(output_geojson_path, driver='GeoJSON')

output_geojson_path


/usr/local/lib/python3.10/site-packages/pyogrio/raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(


'merged_data.geojson'

In [ ]:
import geopandas as gpd

# Load the GeoJSON file
geojson_file_path = 'merged_data.geojson'
geojson_data = gpd.read_file(geojson_file_path)

# Load the Excel file with percentage data
percentage_file_path = 'totalpercentage.xlsx'
percentage_data = pd.read_excel(percentage_file_path)

# Display the first few rows of both datasets to inspect their structure before merging
geojson_data.head(), percentage_data.head()


(   id  OBJECTID  RPO_COD                                         REGPLANORG  \
 0  15        15        1              Capitol Region Council of Governments   
 1  16        16        2    Connecticut Metropolitan Council of Governments   
 2  17        17        3  Lower Connecticut River Valley Council of Gove...   
 3  18        18        4            Naugatuck Valley Council of Governments   
 4  19        19        5    Northeastern Connecticut Council of Governments   
 
          SHORT_NAME  Less than high school graduate  \
 0             CRCOG                           56635   
 1           CTMetro                           25958   
 2          RiverCOG                            5752   
 3  Naugatuck Valley                           36406   
 4            NECCOG                            4824   
 
    High school graduate (includes equivalency)  \
 0                                       177252   
 1                                        57458   
 2                         

In [ ]:
# Use an alternative method to load and inspect the GeoJSON file to work around the previous issue.
import json

# Load the GeoJSON file as raw JSON
with open(geojson_file_path, 'r') as f:
    geojson_data = json.load(f)

# Extract the first feature to inspect the structure and see how to merge
geojson_data['features'][0]['properties']


{'id': 15,
 'OBJECTID': 15,
 'RPO_COD': 1,
 'REGPLANORG': 'Capitol Region Council of Governments',
 'SHORT_NAME': 'CRCOG',
 'Less than high school graduate': 56635,
 'High school graduate (includes equivalency)': 177252,
 "Some college or associate's degree": 161893,
 "Bachelor's degree": 156497,
 'Graduate or professional degree': 126869}

In [ ]:
# Check the structure of the Excel percentage data to identify matching column for the merge
percentage_data.head()


,REGPLANORG,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree,Graduate or professional degree
0,Capitol Region Council of Governments,0.0834,0.2610,0.2384,0.2304,0.1868
1,Connecticut Metropolitan Council of Governments,0.1161,0.2570,0.2180,0.2261,0.1828
2,Lower Connecticut River Valley Council of Gove...,0.0440,0.2466,0.2479,0.2428,0.2187
3,Naugatuck Valley Council of Governments,0.1123,0.2970,0.2776,0.1744,0.1386
4,Northeastern Connecticut Council of Governments,0.0689,0.3627,0.3136,0.1475,0.1073


In [ ]:
# Rename the first column in percentage data for easier merging
percentage_data.rename(columns={'X.100': 'REGPLANORG'}, inplace=True)

# Merge the percentage data with the GeoJSON properties based on the region name
for feature in geojson_data['features']:
    region_name = feature['properties']['REGPLANORG']
    # Find the matching row in the percentage data
    match = percentage_data[percentage_data['REGPLANORG'] == region_name]
    
    if not match.empty:
        # Add the percentage columns to the GeoJSON properties
        feature['properties']['less_than_hs_percentage'] = match['Less than high school graduate'].values[0]
        feature['properties']['hs_graduate_percentage'] = match['High school graduate (includes equivalency)'].values[0]
        feature['properties']['some_college_percentage'] = match["Some college or associate's degree"].values[0]
        feature['properties']['bachelor_degree_percentage'] = match["Bachelor's degree"].values[0]
        feature['properties']['grad_degree_percentage'] = match['Graduate or professional degree'].values[0]

# Save the updated GeoJSON with the merged data
merged_geojson_path = 'updated_merged_data.geojson'
with open(merged_geojson_path, 'w') as f:
    json.dump(geojson_data, f)

merged_geojson_path  # Return the path to the merged file


'updated_merged_data.geojson'